In [16]:
# https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb

In [1]:
import torch
import torch. nn as nn
import torch . optim as optim
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import numpy as np
import spacy
import random
from tensorboardX import SummaryWriter # to print to tensorboard

import inspect

In [2]:
spacy_ger = spacy.load("de_core_news_sm")
spacy_eng = spacy.load("en_core_web_sm")

In [3]:
def tokenizer_ger (text) :
    return [tok. text for tok in spacy_ger.tokenizer(text)]
def tokenizer_eng (text) :
    return [tok. text for tok in spacy_eng.tokenizer(text)]


german = Field(tokenize=tokenizer_ger, lower=True,
               init_token= '<sos>', eos_token= '<eos>' )
english = Field(tokenize=tokenizer_eng, lower=True, 
                init_token= '<sos>', eos_token= '<eos>')

In [11]:
# Pre-process
train_data , test_data, validation_data = Multi30k.splits(exts=('.de', '.en'), fields=(german, english))
vars(train_data.examples[0])

{'src': ['zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.'],
 'trg': ['two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.']}

In [15]:
german.build_vocab(train_data, max_size = 10000, min_freq = 2)
english.build_vocab(train_data, max_size = 10000, min_freq = 2)


print(f"Unique tokens in source (de) vocabulary: {len(german.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(english.vocab)}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, validation_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)



Unique tokens in source (de) vocabulary: 7853
Unique tokens in target (en) vocabulary: 5893


# LSTM Output detail

In [32]:
import ipyplot
images_list = ["img/LSTM1.png" , "img/LSTM2.jpg"]
ipyplot.plot_images(images_list, max_images=10, img_width=400)


In [5]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size,hidden_size, num_layers, dropout):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.dropout = nn.Dropout(dropout)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout)
    
    def forward(self , x): 
        #x.shape : (seq_length, Batch size (N))
        embedding = self.dropout(self.embedding(x))
        output , (hidden,cell) = self.rnn(embedding)
        
        return hidden, cell
        
        

In [16]:
class Decoder(nn.Module): #hid size Encod-Decod must same
    
    pass

In [17]:
class Seq2Seq(nn.Module):
    
    pass

In [29]:
input_state = torch.randn(10000, 1000)
model_base = Encoder(input_size = 10000, embedding_size = 2048 ,hidden_size = 512, num_layers = 2, dropout = 0.5)
model(input_state)

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)

TypeError: forward() missing 1 required positional argument: 'x'